In [115]:
import pandas as pd

# Load the demand profiles CSV
demand_profiles_path = 'C:/Users/noraky/Documents/Masteroppgave/pypsa-earth/resources/demand_profiles.csv'
demand_profiles = pd.read_csv(demand_profiles_path)

# Load the monthly demand profiles Excel
monthly_demand_profiles_path = 'C:/Users/noraky/Documents/Masteroppgave/pypsa-earth/monthly_demand_profiles.xlsx'
monthly_demand_profiles = pd.read_excel(monthly_demand_profiles_path)

# Display the first few rows of each to understand their structure
(demand_profiles.head(), monthly_demand_profiles.head())


(                  time          0         2         4         6         8  \
 0  2013-01-01 00:00:00  12.049224  3.988580  7.840109  3.066470  1.865242   
 1  2013-01-01 01:00:00  11.459934  3.793511  7.456674  2.916499  1.774019   
 2  2013-01-01 02:00:00  10.292872  3.407185  6.697298  2.619487  1.593356   
 3  2013-01-01 03:00:00   9.057806  2.998349  5.893673  2.305169  1.402165   
 4  2013-01-01 04:00:00   8.866117  2.934896  5.768946  2.256385  1.372491   
 
          10         12         14        17  ...        91        92  \
 0  1.506598  10.697193  20.891011  4.783668  ...  3.312962  7.463380   
 1  1.432915  10.174025  19.869295  4.549713  ...  3.150935  7.098369   
 2  1.286989   9.137919  17.845837  4.086377  ...  2.830049  6.375482   
 3  1.132560   8.041438  15.704473  3.596043  ...  2.490464  5.610473   
 4  1.108592   7.871259  15.372122  3.519941  ...  2.437759  5.491739   
 
          93         94         95        96        97         98         99  \
 0  5.5739

In [121]:
from calendar import monthrange
import numpy as np

# Convert time columns to datetime for easier manipulation
demand_profiles['time'] = pd.to_datetime(demand_profiles['time'])
monthly_demand_profiles['time'] = pd.to_datetime(monthly_demand_profiles['time']).dt.month
monthly_demand_profiles.columns = monthly_demand_profiles.columns.astype(str)

print(monthly_demand_profiles['time'])

# Prepare to store the scaling factors for each month and profile
scaling_factors = {}

# For each month in the monthly_demand_profiles
for index, row in monthly_demand_profiles.iterrows():
    month = row['time'].month
    year = row['time'].year
    # Get the range of days for the month to filter the demand profiles
    start_date = pd.Timestamp(year, month, 1)
    end_date = pd.Timestamp(year, month, monthrange(year, month)[1], 23, 59, 59)
    
    # Filter demand_profiles for the current month
    filtered_demand = demand_profiles[(demand_profiles['time'] >= start_date) & (demand_profiles['time'] <= end_date)]
    
    # Sum the hourly loads for each profile in the filtered month
    monthly_sums = filtered_demand.drop('time', axis=1).sum()
    
    # Calculate scaling factors for each profile based on the target monthly total
    for profile in monthly_sums.index:
        if profile in row and monthly_sums[profile] != 0:  # Ensure division is safe
            scaling_factors[(year, month, profile)] = row[profile] / monthly_sums[profile]
        else:  # If no data for the profile, set scaling factor to NaN or 0
            scaling_factors[(year, month, profile)] = 0

# Apply scaling factors to each hour in demand_profiles
adjusted_demand = demand_profiles.copy()
for (year, month, profile), factor in scaling_factors.items():
    if not np.isnan(factor):
        # Only apply scaling where factor is valid
        adjusted_demand[profile] = adjusted_demand.apply(lambda x: x[profile] * factor if x['time'].month == month and x['time'].year == year else x[profile], axis=1)

# Save the adjusted_demand to a new CSV file
adjusted_demand_path = 'C:/Users/noraky/Documents/Masteroppgave/pypsa-earth/custom_files/adjusted_demand_profiles.csv'
adjusted_demand.to_csv(adjusted_demand_path, index=False)

adjusted_demand_path


0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
Name: time, dtype: int32


KeyboardInterrupt: 